In [1]:
pip install sentence-transformers pandas scikit-learn pdfplumber deep-translator pyquery requests lxml openpyxl openai==0.28 python-dotenv transformers torch sentencepiece


  Using cached sentencepiece-0.2.0.tar.gz (2.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for sentencepiece (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [121 lines of output]
      /private/var/folders/s9/np5xjr656bxb0575c963jyvw0000gn/T/pip-build-env-udz1xsbq/overlay/lib/python3.13/site-packages/setuptools/_distutils/dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      /private/var/folders/s9/np5xjr656bxb0575c963jyvw0000gn/T/pip-build-env-udz1xsbq/overlay/lib/python3.13/site-packages/setuptools/dist.py:761: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX 

In [1]:
import os
import pdfplumber
import pandas as pd
from deep_translator import GoogleTranslator
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pyquery import PyQuery as pq
import requests
import json
import tqdm
import openpyxl

/Users/lydialaval/Documents/Data_Sci/Supplier Matching/supplier_matching/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### First Pass

In [ ]:
import os
import json
import requests
import pandas as pd
import pdfplumber
from deep_translator import GoogleTranslator
from pyquery import PyQuery as pq
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# === CONFIGURATION ===
SOURCE_PDF_DIR = '/Users/lydialaval/Documents/Data_Sci/Supplier Matching/sourcing_docs'
SUPPLIER_FILE = '/Users/lydialaval/Documents/Data_Sci/Supplier Matching/SupplierList.xlsx'
COLUMNS_TO_TRANSLATE = ['Description']  # You can add more if needed
LINKEDIN_COLUMN = 'linkedIn'
MODEL_NAME = 'all-MiniLM-L6-v2'

# === LANGUAGE CONGRUENCY ===
def translate_to_english(text):
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception:
        return text  # fallback if translation fails

# === STEP 2: Extract translated text from PDFs into DataFrame ===
def extract_translated_pdfs(pdf_folder):
    records = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            path = os.path.join(pdf_folder, filename)
            with pdfplumber.open(path) as pdf:
                raw_text = "\n".join(page.extract_text() or "" for page in pdf.pages)
                translated = translate_to_english(raw_text)
                records.append({"source_document": filename, "translated_text": translated})
    return pd.DataFrame(records)

# === STEP 3: Load and translate supplier data ===
def load_and_translate_suppliers(filepath):
    df = pd.read_excel(filepath)
    for col in COLUMNS_TO_TRANSLATE:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(translate_to_english)
    return df

# === STEP 4: Scrape LinkedIn About sections ===
def scrape_linkedin_about(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return ""
        d = pq(response.text)
        about = d("section:contains('About')").text()
        return about.strip() if about else ""
    except Exception:
        return ""

# === STEP 5: Combine supplier text (description + LinkedIn About) ===
def enrich_supplier_text(df):
    texts = []
    for _, row in df.iterrows():
        desc = row.get('Description', '')
        linkedin_url = row.get(LINKEDIN_COLUMN, '')
        linkedin_about = scrape_linkedin_about(linkedin_url) if pd.notna(linkedin_url) else ''
        full_text = f"{desc}\n{linkedin_about}".strip()
        texts.append(full_text)
    df['enriched_text'] = texts
    return df

# === STEP 6: Compute similarity and rank for each document ===
def rank_suppliers_per_document(doc_names, doc_texts, supplier_df):
    model = SentenceTransformer(MODEL_NAME)
    supplier_embeddings = model.encode(supplier_df['enriched_text'].tolist())
    
    result = []

    for doc_name, doc_text in zip(doc_names, doc_texts):
        doc_embedding = model.encode([doc_text])[0]
        similarities = cosine_similarity([doc_embedding], supplier_embeddings)[0]

        supplier_df['score'] = similarities
        ranked = supplier_df[['Supplier Name', 'score']].sort_values(by='score', ascending=False).reset_index(drop=True)
        top_suppliers = ranked.head(5).copy()
        top_suppliers['rank'] = top_suppliers.index + 1

        result.append({
            "source_document": doc_name,
            "top_suppliers": top_suppliers[['rank', 'score', 'Supplier Name']].to_dict(orient='records')
        })

    return result, supplier_df

# === MAIN PIPELINE ===
def main():
    print("Extracting and translating sourcing PDFs...")
    pdf_df = extract_translated_pdfs(SOURCE_PDF_DIR)

    print("Loading and translating supplier data...")
    supplier_df = load_and_translate_suppliers(SUPPLIER_FILE)

    print("Scraping LinkedIn and enriching supplier profiles...")
    enriched_supplier_df = enrich_supplier_text(supplier_df)

    print("Ranking suppliers per document...")
    results_json, enriched_supplier_df = rank_suppliers_per_document(
        pdf_df['source_document'].tolist(),
        pdf_df['translated_text'].tolist(),
        enriched_supplier_df
    )

    # Save JSON output
    with open("top_suppliers_by_doc.json", "w") as f:
        json.dump(results_json, f, indent=2)

    print("\nTop suppliers per document saved to: top_suppliers_by_doc.json")

    # Return all key dataframes for inspection
    return results_json, pdf_df, enriched_supplier_df

# === Run the pipeline ===
if __name__ == "__main__":
    results_json, pdf_df, enriched_supplier_df = main()

    # Optional: View in notebook or print
    print("\n=== PDF Document Table ===")
    print(pdf_df.head())

    print("\n=== Enriched Supplier Data ===")
    print(enriched_supplier_df[['Supplier Name', 'score']].head())


### Language congruency, LLM context-aware translation, optional Pyquery automation, document embedding using pretrained transformer, Cosign similarity

In [ ]:
import os
import json
import requests
import pandas as pd
import pdfplumber
from pyquery import PyQuery as pq
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import openai
from dotenv import load_dotenv

# === Load Environment Variables ===
load_dotenv()  # This will load variables from a .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

# === CONFIGURATION ===
SOURCE_PDF_DIR = '/Users/lydialaval/Documents/Data_Sci/Supplier Matching/sourcing_docs'
SUPPLIER_FILE = '/Users/lydialaval/Documents/Data_Sci/Supplier Matching/SupplierList.xlsx'
COLUMNS_TO_TRANSLATE = ['Description','Category','Capability']  # You can add more if needed
LINKEDIN_COLUMN = 'linkedIn'
MODEL_NAME = 'all-MiniLM-L6-v2'

# === STEP 1: Translate text using GPT-4o Mini ===
def translate_to_english(text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Translate the following text into English."},
                {"role": "user", "content": text[:4000]}  # You can limit to 4000 characters
            ]
        )
        translated_text = response.choices[0].message['content'].strip()
        return translated_text
    except Exception as e:
        print(f"Translation error: {e}")
        return text  # fallback if translation fails

# === STEP 2: Extract translated text from PDFs into DataFrame ===
def extract_translated_pdfs(pdf_folder):
    records = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            path = os.path.join(pdf_folder, filename)
            with pdfplumber.open(path) as pdf:
                raw_text = "\n".join(page.extract_text() or "" for page in pdf.pages)
                translated = translate_to_english(raw_text)
                records.append({"source_document": filename, "translated_text": translated})
    return pd.DataFrame(records)

# === STEP 3: Load and translate supplier data ===
def load_and_translate_suppliers(filepath):
    df = pd.read_excel(filepath)
    for col in COLUMNS_TO_TRANSLATE:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(translate_to_english)
    return df

# === STEP 4: Scrape LinkedIn About sections with Pyquery : Not necessary but may be useful to avoid unnecessary human effort ===
def scrape_linkedin_about(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return ""
        d = pq(response.text)
        about = d("section:contains('About')").text()
        return about.strip() if about else ""
    except Exception:
        return ""

# === STEP 5: Combine supplier text (description + LinkedIn About) ===
def enrich_supplier_text(df):
    texts = []
    for _, row in df.iterrows():
        cat = row.get('Category', '')
        cap = row.get('Capability', '')
        linkedin_url = row.get(LINKEDIN_COLUMN, '')
        linkedin_about = scrape_linkedin_about(linkedin_url) if pd.notna(linkedin_url) else ''
        full_text = f"{cat}\n{cap}\n{linkedin_about}".strip()
        texts.append(full_text)
    df['enriched_text'] = texts
    return df

# === STEP 6: Compute similarity and rank for each document ===
def rank_suppliers_per_document(doc_names, doc_texts, supplier_df):
    model = SentenceTransformer(MODEL_NAME)
    supplier_embeddings = model.encode(supplier_df['enriched_text'].tolist())
    
    result = []

    for doc_name, doc_text in zip(doc_names, doc_texts):
        doc_embedding = model.encode([doc_text])[0]
        similarities = cosine_similarity([doc_embedding], supplier_embeddings)[0]

        supplier_df['score'] = similarities
        ranked = supplier_df[['Supplier Id', 'Supplier Name', 'score']].sort_values(by='score', ascending=False).reset_index(drop=True)
        top_suppliers = ranked.head(5).copy()
        top_suppliers['rank'] = top_suppliers.index + 1

        result.append({
            "source_document": doc_name,
            "top_suppliers": top_suppliers[['rank', 'score', 'Supplier Name']].to_dict(orient='records')
        })

    return result, supplier_df

# === MAIN PIPELINE ===
def main():
    print("Extracting and translating sourcing PDFs...")
    pdf_df = extract_translated_pdfs(SOURCE_PDF_DIR)

    print("Loading and translating supplier data...")
    supplier_df = load_and_translate_suppliers(SUPPLIER_FILE)

    print("Scraping LinkedIn and enriching supplier profiles...")
    enriched_supplier_df = enrich_supplier_text(supplier_df)

    print("Ranking suppliers per document...")
    results_json, enriched_supplier_df = rank_suppliers_per_document(
        pdf_df['source_document'].tolist(),
        pdf_df['translated_text'].tolist(),
        enriched_supplier_df
    )

    # Save JSON output
    with open("top_suppliers_by_doc.json", "w") as f:
        json.dump(results_json, f, indent=2)

    supplier_df.to_csv("translated_suppliers.csv", index=False)
    enriched_supplier_df.to_csv("ranked_suppliers.csv", index=False)
    pdf_df.to_csv("pdf_translate.csv", index=False)
    print("\nTop suppliers per document saved to: top_suppliers_by_doc.json")

    # Return all key dataframes for inspection
    return results_json, pdf_df, enriched_supplier_df

# === Run the pipeline ===
if __name__ == "__main__":
    results_json, pdf_df, enriched_supplier_df = main()

    print("\n=== PDF Document Table ===")
    print(pdf_df.head())



CropBox missing from /Page, defaulting to MediaBox


Extracting and translating sourcing PDFs...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [ ]:

print(pdf_df)
pdf_df.to_csv("pdf_translate.csv", index=False)
#enriched_supplier_df[['Supplier Name', 'score']].head()

  source_document                                    translated_text
0    UseCase6.pdf  Sourcing Event: Tender for Acoramidis Folding ...
1    UseCase4.pdf  Title: Flat Gasket Coextrusion  \nCategory: Pl...
2    UseCase5.pdf  Event Name: Call for Tenders for Acoramidis Pa...
3    UseCase1.pdf  Sourcing Event Name: O-Ring 8.00 x 1.00 mm\nPr...
4    UseCase2.pdf  Title: EPP Insulation Stations\nCategories: EP...
5    UseCase3.pdf  Title: Insertion Profile 48x13 Raw/Inserting P...
